In [21]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df=df.dropna()
X=df.drop('label',axis=1)
y=df['label']

In [2]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
voc_size=5000
messages=X.copy()
messages.reset_index(inplace=True)
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [4]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4460 4926 2246]
 [   0    0    0 ...  612  863  388]
 [   0    0    0 ... 1336 2164 3560]
 ...
 [   0    0    0 ... 2737 2104 2630]
 [   0    0    0 ...  618 4246 3048]
 [   0    0    0 ... 1082 4727 3652]]


In [5]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [7]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 34s 18ms/step - loss: 0.4876 - accuracy: 0.7515 - val_loss: 0.2308 - val_accuracy: 0.9026
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1713 - accuracy: 0.9341 - val_loss: 0.2258 - val_accuracy: 0.9100
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1215 - accuracy: 0.9556 - val_loss: 0.2540 - val_accuracy: 0.9064
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0849 - accuracy: 0.9713 - val_loss: 0.2723 - val_accuracy: 0.9067
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0618 - accuracy: 0.9790 - val_loss: 0.3251 - val_accuracy: 0.9006
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0391 - accuracy: 0.9898 - val_loss: 0.3366 - val_accuracy: 0.9007
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0300 - accuracy: 0.9918 - val_loss: 0.4373 - val_accuracy: 0.900

In [8]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90      3419
           1       0.85      0.92      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.89      0.90      0.89      6035
weighted avg       0.90      0.90      0.90      6035



In [27]:
q = model.predict( np.array( [y_test[10],] )  )

In [33]:
print(q)
if q > 0.5:
  print('True news')
else:
  print('False/Fake news')

[[0.4700639]]
False/Fake news
